In [0]:
!wget --no-check-certificate 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQ_GDXRfpLMoxW430-XRe5anOFIyE0NXYU44dLd57Y3GiQzSUciaxg5ep-IZcdVfJx8kqbXSIoM5U85/pub?output=csv' -O w_train.csv

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [0]:
import pandas as pd
train = pd.read_csv('w_train.csv')
train.head(10)

In [0]:
df = pd.DataFrame(train,columns=['tweet','porality'])
df

In [0]:
# Load Pretrained Word2Vec
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [0]:
def get_max_length(df):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['review']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(reviews):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_reviews.append(word2vec_embedding)
    return encoded_reviews
        
def get_padded_encoded_reviews(encoded_reviews):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def sentiment_encode(sentiment):
    """
    return one hot encoding for Y value
    """
    if sentiment == 'positive':
        return [1,0]
    else:
        return [0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    reviews = df['review'].tolist()
    
    encoded_reviews = get_word2vec_enc(reviews)
    padded_encoded_reviews = get_padded_encoded_reviews(encoded_reviews)
    # encoded sentiment
    sentiments = df['sentiment'].tolist()
    encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_sentiment)
    return X, Y 

In [0]:
def AddFormat(text,sentiment,reviews_train):
  format = {'review': text,'sentiment': sentiment}
  reviews_train.append(format)
  return 0

In [0]:
reviews_train = []
data = np.array(df)
for i in data :
  AddFormat(i[0],i[1],reviews_train)

df = pd.DataFrame(reviews_train)

print(reviews_train)
# max_length is used for max sequence of input
max_length = get_max_length(df)

train_X, train_Y = preprocess(df)

In [0]:
# LSTM model
model = Sequential()
model.add(LSTM(32))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
print('Train...')
model.fit(train_X, train_Y,epochs=50)

In [0]:
model.summary()

In [0]:
cnn_reviews_test = [
         {'review': 'needs to learn to share the desk  and  have a lot to add', 'sentiment': 'positive'},
         {'review': 'everyone can be a hero.', 'sentiment': 'positive'},
         {'review': 'i think its for to learn to analyze rather than blindly believe', 'sentiment': 'positive'},
         {'review': 'russia lost 27 ka52 attack helicopters in the ukraine invasion', 'sentiment': 'negative'},
         {'review': 'card games are boring so how did two of them become my favourites of 2022.', 'sentiment': 'negative'},
         {'review': 'fuck', 'sentiment': 'negative'}
    ]
test_df = pd.DataFrame(cnn_reviews_test)

test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)